<a href="https://colab.research.google.com/github/kili-technology/automl/blob/main/notebooks/semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will see how we can simply create a segmentation model with AutoML to pre-annotate our dataset on the Kili Platform.

In [ ]:
from getpass import getpass
API_KEY=getpass("Enter your KILI API key: ")
%env KILI_API_KEY=$API_KEY

# Installation AutoML (5min)

We first follow the install procedure explained in the README.md.

In [ ]:
!git clone https://github.com/kili-technology/automl.git

In [ ]:
%cd automl
!git submodule update --init

In [ ]:
# %pip install torch
%pip install -e .

In [ ]:
import os
os.environ["PYTHONPATH"] += ":/content/automl/"

In [ ]:
KILI_URL="https://cloud.kili-technology.com/"

os.environ['api_endpoint'] =f'{KILI_URL}api/label/v2/graphql'
api_endpoint = os.environ['api_endpoint']

In [ ]:
USE_DRIVE=False
if USE_DRIVE:
  from google.colab import drive
  drive.mount("/content/drive")

  os.environ['KILIAUTOML_CACHE'] ="/content/drive/automl"
else:
  os.environ['KILIAUTOML_CACHE'] = os.path.join(os.environ['HOME'], ".cache", "kili", "automl")

KILIAUTOML_CACHE = os.environ['KILIAUTOML_CACHE']

# Setup Mock Coco Project

Here we download a dataset in the coco format and we convert it to the kili format before uploading it to Kili

In [ ]:
!wget https://storage.googleapis.com/kili-machine-learning-automl/notebooks/coco_dataset_2017_val.zip
!unzip -q coco_dataset_2017_val.zip 
dataset_dir = "./coco_dataset"

#### Convert coco format to kili format
Then we use the kili format to create a new projet containing those coco images.

In [ ]:
LIMIT = 200 # nb of images

In [ ]:
from typing import Dict, List
import numpy as np
from tqdm.autonotebook import tqdm

from typing_extensions import Literal, TypedDict

class CategoryT(TypedDict):
    name: str
    confidence: int  # between 0 and 100

# ## DETECTRON FORMAT

class ImageCoco(TypedDict):
    id: int
    license: int
    file_name: str
    height: int
    width: int
    date_captured: None


class CategoryCoco(TypedDict):
    id: int
    name: str
    supercategory: str


class AnnotationsCoco(TypedDict):
    id: int
    image_id: int  # -> external_id : the last part of the url
    category_id: int
    bbox: List[int]
    segmentation: List[List[float]]  # [[x, y, x, y, x ...]]
    area: int
    iscrowd: int


class CocoFormat(TypedDict):
    info: Dict  # type: ignore
    licenses: List[Dict]  # type: ignore
    categories: List[CategoryCoco]
    images: List[ImageCoco]
    annotations: List[AnnotationsCoco]


# ## KILI Polygon Semantic Format

class NormalizedVertice(TypedDict):
    x: float
    y: float


class NormalizedVertices(TypedDict):
    normalizedVertices: List[NormalizedVertice]


class SemanticAnnotation(TypedDict):
    boundingPoly: List[NormalizedVertices]  # len(self.boundingPoly) == 1
    mid: str
    type: Literal["semantic"]
    categories: List[CategoryT]


class SemanticJob(TypedDict):
    annotations: List[SemanticAnnotation]


job_name = "SEMANTIC_JOB"


def camelCase(st):
    return st.capitalize().replace(" ", "_")

def convert_coco_to_kili(coco_format: CocoFormat) -> Dict[str, SemanticJob]:
    """
    Coco format:
    <dataset_dir>/
        data/
            <filename0>.<ext>
            <filename1>.<ext>
            ...
        labels.json

    We convert the json to kili format.
    """
    mapping_external_id_to_semanticjob: Dict[str, SemanticJob] = {}

    print("Nb categories", len(coco_format["categories"]))
    print("Nb annotations", len(coco_format["annotations"]))
    print("Nb images", len(coco_format["images"]))

    for coco_annotation in tqdm(coco_format["annotations"], desc="Extracting COCO Objects"):
        # Extract Coco info
        category_names = [
            cat["name"] for cat in coco_format["categories"] if cat["id"] == coco_annotation["category_id"]
        ]
        assert len(category_names) == 1
        category_name = category_names[0]
        category_kili_id = camelCase(category_name)

        image_names = [
            image
            for image in coco_format["images"]
            if image["id"] == coco_annotation["image_id"]
        ]
        assert len(image_names) == 1
        external_id = image_names[0]["file_name"]
        height, width = image_names[0]["height"], image_names[0]["width"]

        # convert to Kili
        # Each connected component becones a new object in Kili format
        connected_components  : List[SemanticAnnotation]= []
        for single_connected_component in coco_annotation["segmentation"]:
            tab_xy = single_connected_component  # We take only the first connected component
            if type(tab_xy) != list:
                # print(single_connected_component)
                continue
            tab_x = list(np.array(tab_xy[::2]) / width )
            tab_y = list(np.array(tab_xy[1::2]) / height )

            normalizedVertices: NormalizedVertices = {
                "normalizedVertices": [NormalizedVertice(x=x, y=y) for x, y in zip(tab_x, tab_y)]
            }
            boundingPoly = [normalizedVertices]
            categories = [CategoryT(name=category_kili_id, confidence=100)]

            annotation_kili = SemanticAnnotation(
                boundingPoly=boundingPoly,
                mid=None,# type:ignore  # Created on the fly
                type="semantic",
                categories=categories,
            )
            connected_components.append(annotation_kili)
        if external_id not in mapping_external_id_to_semanticjob:
            mapping_external_id_to_semanticjob[external_id] = SemanticJob(annotations=connected_components)
        else:
            previous_annotatations = mapping_external_id_to_semanticjob[external_id]["annotations"]
            mapping_external_id_to_semanticjob[external_id] = SemanticJob(annotations=previous_annotatations + connected_components)

    return mapping_external_id_to_semanticjob


def convert_coco_to_kili_json_interface(coco_format: CocoFormat):
    """
    Coco format:
    <dataset_dir>/
        data/
            <filename0>.<ext>
            <filename1>.<ext>
            ...
        labels.json

    We convert the json to kili format.
    """
    coco_categories = coco_format["categories"]

    import random

    number_of_colors = len(coco_categories)

    colors = [
        "#" + "".join([random.choice("0123456789ABCDEF") for __ in range(6)])
        for _ in range(number_of_colors)
    ]


    categories = {
        camelCase(cat["name"]): {
            "children": [],
            "name": cat["name"],
            "color": color,
            "id": cat["id"],
        }
        for cat, color in zip(coco_categories, colors)
    }

    json_interface = {
        "jobs": {
            job_name: {
                "content": {"categories": categories, "input": "radio"},
                "instruction": "Categories",
                "isChild": False,
                "tools": ["semantic"],
                "mlTask": "OBJECT_DETECTION",
                "models": {"interactive-segmentation": {"job": job_name + "_MARKER"}},
                "isVisible": True,
                "required": 1,
                "isNew": False,
            },
            job_name + "_MARKER": {
                "content": {"categories": categories, "input": "radio"},
                "instruction": "Categories",
                "isChild": False,
                "tools": ["marker"],
                "mlTask": "OBJECT_DETECTION",
                "isModel": True,
                "isVisible": False,
                "required": 0,
                "isNew": False,
            },
        }
    }

    return json_interface


In [ ]:
from kili.client import Kili
import json

with open(dataset_dir + "/raw/instances_val2017.json", "r") as f:
    coco_format = json.load(f)


json_interface = convert_coco_to_kili_json_interface(coco_format=coco_format)

assets = [
    {
        "externalId": asset["file_name"],
        "content": dataset_dir + "/validation/data/" + asset["file_name"],
        "metadata": {},
    }
    for asset in coco_format["images"][:LIMIT]
]


kili = Kili(api_endpoint=api_endpoint)

# Create project
project = kili.create_project(
    input_type="IMAGE",
    json_interface=json_interface,
    title="Coco to Kili",
    description="",
    project_type=None,
)
project_id = project["id"] # type:ignore



#### Add assets and labels to assets

We add labels to half of the data to simulate a project where we haven't labeled much data and we want to predict the labels of the unlabeled data.

In [ ]:
# Add assets
print(f"Uploading {len(assets)} images to Kili")
external_id_array = [a.get("externalId") for a in assets]
content_array = [a.get("content") for a in assets]
json_metadata_array = [a.get("metadata") for a in assets]
kili.append_many_to_dataset(
    project_id=project_id,
    content_array=content_array,# type:ignore
    external_id_array=external_id_array,# type:ignore
    json_metadata_array=json_metadata_array,# type:ignore
)


# Add labels to half the assets
from tqdm import tqdm
mapping_external_id_to_semanticjob = convert_coco_to_kili(coco_format=coco_format)
asset_ids = kili.assets(project_id=project_id, fields=["id", "externalId"], first=int(LIMIT/2))
asset_ids = list(asset_ids)
print(f"Labelling {len(asset_ids)} images in Kili")
for i, asset_id in tqdm(enumerate(asset_ids), total=len(asset_ids)):
    external_id = asset_id["externalId"]

    if external_id in mapping_external_id_to_semanticjob:
        semantic_job = mapping_external_id_to_semanticjob[external_id]

        # print(f"Nb annotation on image {external_id}", len(semantic_job["annotations"]))
        kili.append_to_labels(
            label_asset_id=asset_id["id"],
            json_response={"SEMANTIC_JOB": SemanticJob(annotations=semantic_job["annotations"])},
        )
    else:
        print("Warning: No Annotation on image", external_id)


In [ ]:
# COCO
project_id = project_id
job_name = "SEMANTIC_JOB"

# 2 cars (fast demo)
# project_id = "cl4cisaq36awx0lpb8ql57mxk"
# job_name = "JOB_0"

# Training an object detection model with Kiliautoml

The following command will automatically download the labeled data in your Kili project. Then, it will choose the right model for Object detection, train it with this data and save it locally. You can visualize the training evolution on Tensorboard.

In [ ]:
!rm -rf $KILIAUTOML_CACHE
!kiliautoml train --project-id $project_id --epochs 30 --target-job $job_name --api-endpoint $api_endpoint --batch-size 10

We just finetuned the model with 100 images for 200 categories. Some categories like scissors have not even been seen by the model. SOTA average precision (AP) is around 60. Here, we trained the model in 7 minutes, but you will need to use more images/raise the number of epochs in production.

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
TENSOR_BOARD=f"{KILIAUTOML_CACHE}/{project_id}/{job_name}/detectron2/pytorch/model"
%tensorboard --logdir $TENSOR_BOARD

# Preannotate and send predictions


Now we can use our local trained model to predict the classes of our text assets and send the prediction scores to the project on Kili. These preannotations can then be validated or corrected by annotators.

In [ ]:
!kiliautoml predict --project-id $project_id  --target-job $job_name  --api-endpoint $api_endpoint --max-assets 21000

Now you can ckeck that your assets have predictions on Kili!

In [ ]:
print(f"{KILI_URL}label/projects/{project_id}/menu/queue?currentPage=1&pageSize=20")

In [ ]:
from IPython.display import Image
import glob

print(f'{KILIAUTOML_CACHE}/{project_id}/{job_name}/detectron2/data/*')

files = glob.glob(f'{KILIAUTOML_CACHE}/{project_id}/{job_name}/detectron2/data/*')
Image(files[1])